In [ ]:
#crea una funcion que recib un año, y retorne un dataframe a partir de df_latam cuya primera columna es el pais, y el resto de columnas son los valores de todos los indicadores en el año que ingresa a la funcion
def obtener_dataframe_por_anio(df, anio):   
    # Filtrar el DataFrame para el año específico
    df_anio = df[['Country Name', 'region', 'Indicator Name', anio]].copy()

    # Renombrar la columna del año a 'Value'
    df_anio.rename(columns={anio: 'Value'}, inplace=True)

    # Pivotar el DataFrame para que los indicadores sean columnas
    df_pivot = df_anio.pivot(index='Country Name', columns='Indicator Name', values='Value').reset_index()

    return df_pivot
# Llamar a la función para obtener el DataFrame para un año específico  
anio = '2014'
df_anio = obtener_dataframe_por_anio(df_latam, anio)


# transforma los valores faltantes todos en -1 para df_anio
df_anio.fillna(-1, inplace=True)

# me gustaria visualizar esta data en un plot 2D usando pca. considera que los features son los indicadores y en el grafico pon las siglas del pais
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# Normalizar los datos
scaler = MinMaxScaler()
df_anio_scaled = scaler.fit_transform(df_anio.drop(columns=['Country Name']))
# Aplicar PCA
pca = PCA(n_components=2)
df_anio_pca = pca.fit_transform(df_anio_scaled)
# Crear un DataFrame con los resultados de PCA
df_pca = pd.DataFrame(data=df_anio_pca, columns=['PCA1', 'PCA2'])
df_pca['Country Name'] = df_anio['Country Name']
# Graficar los resultados de PCA
plt.figure(figsize=(10, 8))
sns.scatterplot(data=df_pca, x='PCA1', y='PCA2')
for i in range(df_pca.shape[0]):
    plt.text(df_pca['PCA1'][i], df_pca['PCA2'][i], df_pca['Country Name'][i])
plt.title(f'PCA de Indicadores en {anio}')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.grid()
plt.show()
# Imprime los valores de PCA
print(df_pca)

# imprime la correlacion entre los componentes principales y los indicadores de mayor a menor
pca_components = pca.components_
df_correlacion = pd.DataFrame(data=pca_components, columns=df_anio.drop(columns=['Country Name']).columns)
df_correlacion = df_correlacion.transpose()
df_correlacion.columns = ['Componente Principal 1', 'Componente Principal 2']
df_correlacion['Correlacion'] = np.abs(df_correlacion['Componente Principal 1']) + np.abs(df_correlacion['Componente Principal 2'])
df_correlacion = df_correlacion.sort_values(by='Correlacion', ascending=False)
print(df_correlacion)
